# Moneytree port for expenses (v1)

## Prerequisites
The input file must be tab separated (tsv). Columns must be in the following order
TransactionDate Vendor TransactionAmount ExpenseCategory Notes(optional). No column headers.

### Set file name

In [ ]:
file="expenses"

### Remove carriage return character in expenses. This is from Excel. Run this to be safe

In [ ]:
sed -i 's/\r$//' expenses
echo "done"

### Verify file is tab separated

In [ ]:
head -n 5 "$file"

### OPTIONAL: if you copied the filed over from a Windows environment into a linux (jupyter) environment, you may convert the file to UTF8 so you can open it on jupyter.

In [ ]:
iconv -f iso-8859-1 -t UTF-8 "$file" -o "$file"

### Set host. Usually local IP to communicate with server

In [ ]:
host=""

### Check server is running
Expected output: "Money tree is healthy"

In [ ]:
curl "http://$host:9000/health"

### Collect unique vendors

In [ ]:
awk -F "\t" ' !seen[$2]++ { print $2 } ' expenses >> vendors
echo "done"

### Construct post-vendors for CURL payload to add them to the database

In [ ]:
awk -F "\t" '{ printf("{ \"name\": \"%s\" }\n", $1) }' vendors >> post-vendors
echo "done"

### Confirm first couple lines look normal
Expected: { "name": "vendorName" }

In [ ]:
head -n 3 post-vendors

### Make a POST request for each json to add Vendors to database and collect the responses for the UUIDs as vendors-full

In [ ]:
while IFS= read -r line
do
   curl -w '\n' -H "Content-Type: application/json" -sX POST -d "$line" "http://$host:9000/vendor" >> vendors-full
done < post-vendors
echo "done"

### Verify the first couple response jsons look normal

In [ ]:
head -n 3 vendors-full 

### Construct Vendor's key value storage

In [ ]:
declare -A vendorsKv

while IFS= read -r line
do
    id=$(echo "$line" | jq -r .id)
    name=$(echo "$line" | jq -r .name)
    vendorsKv["$name"]="$id"
done < vendors-full
echo "done"

### Collect unique expense category

In [ ]:
awk -F "\t" ' !seen[$4]++ { print $4 } ' expenses >> expense-category
echo "done"

### Construct post-expense-category for CURL payload to add them to the database

In [ ]:
awk -F "\t" '{ printf("{ \"name\": \"%s\", \"targetAmount\": 0.00 }\n", $1) }' expense-category >> post-expense-category
echo "done"

### Confirm first couple lines look normal
Expected: { "name": "categoryName", "targetAmount": 0.00 }

In [ ]:
head -n 3 post-expense-category

### Make a POST request for each json to add Expense Category to database and collect the responses for the UUIDs as expense-category-full

In [ ]:
while IFS= read -r line
do
   curl -w '\n' -H "Content-Type: application/json" -sX POST -d "$line" "http://$host:9000/category/expense" >> expense-category-full
done < post-expense-category
echo "done"

### Verify the first couple response jsons look normal

In [ ]:
head -n 3 expense-category-full

### Construct Expense Categpory's key value storage

In [ ]:
declare -A expenseCategoryKv

while IFS= read -r line
do
    id=$(echo "$line" | jq -r .id)
    name=$(echo "$line" | jq -r .name)
    expenseCategoryKv["$name"]="$id"
done < expense-category-full
echo "done"

# IMPORTANT: If the transaction is NOT in ISO-8601 format, ie yyyy-mm-dd, run the next cell. OTHERWISE SKIP TO NEXT STEP

In [ ]:
awk -F "\t" '{ split($1,date,"-"); printf("%s-%s-%s\t%s\t%s\t%s\t%s\n",date[3], date[1], date[2],$2,$3,$4,$5) }' expenses >> expenses-iso-dated
echo "done"
file="expenses-iso-dated"

### Generate POST json payload for expenses

In [ ]:
while IFS=$'\t' read -r date vendor amount category notes
do
    echo "{ \"transactionDate\": \"$date\", \"transactionAmount\": $amount, \"vendor\": \"${vendorsKv["$vendor"]}\", \"expenseCategory\": \"${expenseCategoryKv["$category"]}\", \"notes\": \"$notes\", \"hide\": false }" >> post-expenses
done < "$file"
echo "done"

### Verify payload looks normal

In [ ]:
head -n 3 post-expenses

### Insert the expenses

In [ ]:
while IFS= read -r line
do
   curl -H "Content-Type: application/json" -o /dev/null -sX POST -d "$line" "http://$host:9000/expense" 
done < post-expenses
echo "done"

# DONE!